In [1]:
import keras


In [2]:
import numpy as np


In [3]:
import tensorflow

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy as np
import os
import cv2


In [6]:
json_file = open('modell/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)

In [7]:
model.load_weights("modell/model.h5")
print("Loaded model from disk")

Loaded model from disk


In [8]:
from PIL import Image

In [9]:
image = cv2.imread('t1.png')
image= cv2.resize(image, (30,30))
# (image)

In [10]:
d=[]
d.append(image)
t=np.array(d)
p= model.predict_classes(t)
p

c:\users\akshat\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([7], dtype=int64)

In [11]:
cv2.imshow('s',image)
cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
data=[]
image_fromarray = Image.fromarray(image, 'RGB')
resize_image = image_fromarray.resize((30,30))
data.append(np.array(resize_image))

X_test = np.array(data)
X_test = X_test/255

pred = model.predict_classes(X_test)

In [13]:
int(pred)

7

In [2]:
import cv2
import math
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
json_file = open('modell/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("modell/model.h5")
print("Loaded model from disk")

def getcontours(img):
    
    img = cv2.cvtColor ( img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur ( img , (3,3) , 2)
    img = cv2.Canny ( img , 100 , 50)
    objtype=0
    aa=[]
    contours_cirles = []
    contours, hierarchy = cv2.findContours( img , cv2.RETR_TREE , cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
                
        if 250<area<150000 :
#             cnt = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernal)
            perimeter = cv2.arcLength( cnt, True)
            approx = cv2.approxPolyDP( cnt, 0.01*perimeter, True)
            objcorn = len(approx)
            x, y, w, h = cv2.boundingRect(approx)
            a= [x, y, x+w, y+h]
            circularity = 4*math.pi*(area/(perimeter*perimeter))

            if (0.7 < circularity < 1.3) and (10<objcorn<23) :
                contours_cirles.append(cnt)
                objtype='circle'
                cv2.rectangle( frame, (x,y), (x+w,y+h), (0,255,5), 1)
                cv2.drawContours( final_result, cnt, -1, (0,255,0), 2)
                cv2.putText( frame , objtype ,((x+w//2)-10,(y+h//2)-10) , cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1)
                aa.append(a)
                
            else:
                pass
    return aa
                 
#-------------------------------------------------------------------------------------------------------------#
w=320
h=240

# for n in range(1,15):
#     kernel = np.ones((5,5), np.uint8)
#     frame = cv2.imread(str(n)+'.jpg')
#     img = frame.copy()
#     img = cv2.resize(img,(640,480))

#     frame = cv2.resize(frame,(640,480))
#     imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     while(True):

cap = cv2.VideoCapture('testvideo101.mkv')

while(cap.isOpened()):
        ret, frame = cap.read()
        img = frame.copy()
        frame = cv2.resize(frame,(640,480))
        img=cv2.resize(img,(640,480))
        imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
        red_lower = np.array([0, 110, 60 ], np.uint8) 
        red_upper = np.array([15, 255, 255], np.uint8) 
        red_mask = cv2.inRange(imgHSV, red_lower, red_upper) 
        red_lower1 = np.array([160, 110, 60 ], np.uint8) 
        red_upper1 = np.array([179, 255, 255], np.uint8) 
        red_mask1 = cv2.inRange(imgHSV, red_lower1, red_upper1) 
        kernal = np.ones((3,3), "uint8") 
        red_mask = cv2.dilate(red_mask, kernal) 
        red_mask1 = cv2.dilate(red_mask1, kernal) 
        imgresult = cv2.bitwise_and(frame, frame, mask = red_mask)
        imgresult1 = cv2.bitwise_and(frame,frame, mask = red_mask1)
        final_result = cv2.add(imgresult, imgresult1)
        final_result = cv2.GaussianBlur(final_result, (3,3), 0)
        final_result = cv2.dilate(final_result, kernal,4)
        final_result = cv2.erode(final_result, kernal,4)

        points = getcontours(final_result)
        
        cv2.imshow("frame",frame)
        cv2.imshow('final_result',final_result)
        
        try:
                final_image=[]

                ROI = img[(points[0][1]-5) : (points[0][3]+6), (points[0][0]-6) : (points[0][2]+6)]

                ROI = cv2.resize(ROI, (200,200))
                ITBC = cv2.resize(ROI, (30,30))
                final_image.append(ITBC)
                img_for_det = np.array(final_image)
                prediction = model.predict_classes(img_for_det)
#                 print(int(prediction))
               
                if (int(prediction))<9:
                    print(int(prediction))
                    cv2.imshow('ROI', ROI)
                    cv2.putText(ROI, int(prediction)  ,((points[0][1]),(points[0][0])) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

                
                

        
        except:
            pass
        
        ROI=0
        
#         cv2.imshow("frame",frame)
#         cv2.imshow('final_result',final_result)

        if cv2.waitKey(1) & 0xFF == ord(' '):
            break
cap.release()
cv2.destroyAllWindows()    


Loaded model from disk


c:\users\akshat\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [30]:
import cv2
import math
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
json_file = open('modell/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("modell/model.h5")
print("Loaded model from disk")

def getcontours(img):
    
    img = cv2.cvtColor ( img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur ( img , (3,3) , 2)
    img = cv2.Canny ( img , 100 , 50)
    objtype=0
    aa=[]
    contours_cirles = []
    contours, hierarchy = cv2.findContours( img , cv2.RETR_TREE , cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
                
        if 350<area<150000 :
#             cnt = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernal)
            perimeter = cv2.arcLength( cnt, True)
            approx = cv2.approxPolyDP( cnt, 0.01*perimeter, True)
            objcorn = len(approx)
            x, y, w, h = cv2.boundingRect(approx)
            a= [x, y, x+w, y+h]
            circularity = 4*math.pi*(area/(perimeter*perimeter))

            if (0.7 < circularity < 1.3) and (10<objcorn<23) :
                contours_cirles.append(cnt)
                objtype='circle'
                cv2.rectangle( frame, (x,y), (x+w,y+h), (0,255,5), 1)
                cv2.drawContours( final_result, cnt, -1, (0,255,0), 2)
                cv2.putText( frame , objtype ,((x+w//2)-10,(y+h//2)-10) , cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1)
                aa.append(a)
                
            else:
                pass
    return aa
                 
#-------------------------------------------------------------------------------------------------------------#
w=320
h=240

for n in range(1,15):
    kernel = np.ones((5,5), np.uint8)
    frame = cv2.imread(str(n)+'.jpg')
    img = frame.copy()
    img = cv2.resize(img,(640,480))

    frame = cv2.resize(frame,(640,480))
    imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    while(True):

# cap = cv2.VideoCapture('testvideo101.mkv')

# while(cap.isOpened()):
#         ret, frame = cap.read()
#         img = frame.copy()
#         frame = cv2.resize(frame,(640,480))
#         img=cv2.resize(img,(640,480))
#         imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
        red_lower = np.array([0, 110, 60 ], np.uint8) 
        red_upper = np.array([15, 255, 255], np.uint8) 
        red_mask = cv2.inRange(imgHSV, red_lower, red_upper) 
        red_lower1 = np.array([160, 110, 60 ], np.uint8) 
        red_upper1 = np.array([179, 255, 255], np.uint8) 
        red_mask1 = cv2.inRange(imgHSV, red_lower1, red_upper1) 
        kernal = np.ones((3,3), "uint8") 
        red_mask = cv2.dilate(red_mask, kernal) 
        red_mask1 = cv2.dilate(red_mask1, kernal) 
        imgresult = cv2.bitwise_and(frame, frame, mask = red_mask)
        imgresult1 = cv2.bitwise_and(frame,frame, mask = red_mask1)
        final_result = cv2.add(imgresult, imgresult1)
        final_result = cv2.GaussianBlur(final_result, (3,3), 0)
        final_result = cv2.dilate(final_result, kernal,4)
        final_result = cv2.erode(final_result, kernal,4)

        points = getcontours(final_result)
        
        cv2.imshow("frame",frame)
        cv2.imshow('final_result',final_result)
        
        try:
                final_image=[]

                ROI = img[(points[0][1]-5) : (points[0][3]+6), (points[0][0]-6) : (points[0][2]+6)]

                ROI = cv2.resize(ROI, (200,200))
                ITBC = cv2.resize(ROI, (30,30))
                final_image.append(ITBC)
                img_for_det = np.array(final_image)
                prediction = model.predict_classes(img_for_det)
               
               
                if (int(prediction))<9:
                    print(int(prediction))
                    cv2.imshow('ROI', ROI)
                    cv2.putText(ROI, int(prediction)  ,((points[0][1]),(points[0][0])) ,
                                cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)

        except:
            pass
        
        ROI=0
        
#         cv2.imshow("frame",frame)
#         cv2.imshow('final_result',final_result)

        if cv2.waitKey(1) & 0xFF == ord(' '):
            break
# cap.release()
cv2.destroyAllWindows()    


Loaded model from disk


In [ ]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

#---------------------------------------------------------------------------------------------------------------#
import cv2
import math
import numpy as np
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json

json_file = open('modell/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)
model.load_weights("modell/model.h5")
print("\t\tLoaded model from disk")

def getcontours(img):
    
    img = cv2.cvtColor ( img, cv2.COLOR_BGR2GRAY)
    img = cv2.GaussianBlur ( img , (3,3) , 2)
    img = cv2.Canny ( img , 100 , 50)
    objtype=0
    aa=[]
    contours_cirles = []
    contours, hierarchy = cv2.findContours( img , cv2.RETR_TREE , cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
                
        if 350<area<15000000 :
#             cnt = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernal)
            perimeter = cv2.arcLength( cnt, True)
            approx = cv2.approxPolyDP( cnt, 0.01*perimeter, True)
            objcorn = len(approx)
            x, y, w, h = cv2.boundingRect(approx)
            a= [x, y, x+w, y+h]
            circularity = 4*math.pi*(area/(perimeter*perimeter))

            if (0.7 < circularity < 1.3) and (10<objcorn<23) :
                contours_cirles.append(cnt)
                objtype='circle'
                #cv2.rectangle( frame, (x,y), (x+w,y+h), (0,255,5), 1)
                cv2.drawContours( final_result, cnt, -1, (0,255,0), 2)
              #  cv2.putText( frame , objtype ,((x+w//2)-10,(y+h//2)-10) , cv2.FONT_HERSHEY_COMPLEX, 0.5, (0,255,0), 1)
                aa.append(a)
                
            else:
                pass
    return aa
                 
#-------------------------------------------------------------------------------------------------------------#
w=320
h=240

for n in range(1,11):
    kernel = np.ones((5,5), np.uint8)
    frame = cv2.imread('w'+str(n)+'.jpg')
    img = frame.copy()
    img = cv2.resize(img,(640,480))

    frame = cv2.resize(frame,(640,480))
    imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    while(True):

# cap = cv2.VideoCapture('testvideo101.mkv')

# while(cap.isOpened()):
#         ret, frame = cap.read()
#         img = frame.copy()
#         frame = cv2.resize(frame,(640,480))
#         img=cv2.resize(img,(640,480))
#         imgHSV = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
        red_lower = np.array([0, 110, 60 ], np.uint8) 
        red_upper = np.array([15, 255, 255], np.uint8) 
        red_mask = cv2.inRange(imgHSV, red_lower, red_upper) 
        
        red_lower1 = np.array([160, 110, 60 ], np.uint8) 
        red_upper1 = np.array([179, 255, 255], np.uint8) 
        red_mask1 = cv2.inRange(imgHSV, red_lower1, red_upper1) 
        
        kernal = np.ones((3,3), "uint8") 
        
        red_mask = cv2.dilate(red_mask, kernal) 
        red_mask1 = cv2.dilate(red_mask1, kernal) 
        
        imgresult = cv2.bitwise_and(frame, frame, mask = red_mask)
        imgresult1 = cv2.bitwise_and(frame,frame, mask = red_mask1)
        
        final_result = cv2.add(imgresult, imgresult1)
        final_result = cv2.GaussianBlur(final_result, (3,3), 0)
        
        final_result = cv2.dilate(final_result, kernal,4)
        final_result = cv2.erode(final_result, kernal,4)

        points = getcontours(final_result)
        
        cv2.imshow("Original Image",frame)
        cv2.imshow('processed Image',final_result)
        
        try:
                final_image=[]

                ROI = img[(points[0][1]-5) : (points[0][3]+6), (points[0][0]-6) : (points[0][2]+6)]

                ROI = cv2.resize(ROI, (250,250))
                sign_board = ROI.copy()
                ITBC = cv2.resize(ROI, (30,30))
                final_image.append(ITBC)
                img_for_det = np.array(final_image)
                prediction = model.predict_classes(img_for_det)
                class_no = int(prediction)
                class_name=classes[class_no]
               
               
                if (int(prediction))<9:
                    print('\t'+ class_name)
                    cv2.putText(sign_board, class_name, (10,26), cv2.FONT_HERSHEY_COMPLEX, 0.6, (0,255,250), 2)

                    cv2.imshow('Sign', sign_board)
#                    cv2.putText(ROI, class_name, ((points[0][1]),(points[0][0])) ,
 #                               cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                 
        except:
            pass
        
        ROI=0
        
#         cv2.imshow("frame",frame)
#         cv2.imshow('final_result',final_result)

        if cv2.waitKey(10) & 0xFF == ord(' '):
            break
            
# cap.release()
cv2.destroyAllWindows()    


		Loaded model from disk
	Speed limit (100km/h)
	Speed limit (100km/h)
	Speed limit (100km/h)
	Speed limit (80km/h)
	Speed limit (80km/h)
	Speed limit (80km/h)
	Speed limit (100km/h)
	Speed limit (100km/h)
	Speed limit (100km/h)
	Speed limit (100km/h)
	Speed limit (70km/h)
	Speed limit (70km/h)
	Speed limit (70km/h)
	Speed limit (70km/h)
	Speed limit (70km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
	Speed limit (50km/h)
